# 1. Хеширование

Хеширование — это процесс преобразования входных данных (обычно строки) в фиксированный набор байтов (хеш-значение) с помощью хеш-функции. 

Основные характеристики хеш-функций:
1) Однонаправленность : Из хеш-значения невозможно восстановить исходные данные.
2) Быстрота вычисления : Хеш-функция должна быстро вычислять хеш-значение для любого входного данных.
3) Совпадение минимальное (коллизии) : Вероятность того, что два разных входных данных имеют одинаковое хеш-значение, должна быть минимальной.
4) Детерминированность : Для одинаковых входных данных хеш-функция всегда должна возвращать одинаковое хеш-значение.

# Алгоритм Рабина-Карпа.
Алгоритм Рабина-Карпа — это эффективный алгоритм поиска подстроки в строке. Он использует хеширование для быстрого сравнения подстрок.

Основные идеи алгоритма:
1) Хеширование подстроки и окна : Вычисляется хеш-значение искомой подстроки и каждого окна в строке одинаковой длины.
2) Сравнение хеш-значений : Если хеш-значения совпадают, выполняется дополнительное сравнение символов для подтверждения совпадения.
3) Смещение окна : Окно смещается на один символ вправо, и хеш-значение обновляется.
Преимущества алгоритма:
1) Эффективность : Средняя временная сложность — O(n + m), где n — длина строки, m — длина подстроки.

In [ ]:
def polhash(data, a, b):
    res = 0
    for x in data:
        res = (res * a + ord(x) - ord('a') + 1) % b
    return res

In [ ]:
def code(symbol):
    """Преобразует символ в числовое значение (a->1, b->2, ..., z->26)"""
    return ord(symbol) - ord('a') + 1  # ord('a') = 97, но конкретное значение не важно, так как важно только разность

def poly_hash(string):
    """
    Вычисляет полиномиальный хеш для всех префиксов строки
    Возвращает массив, где hash[i] - хеш для string[0..i-1]
    """
    values = [0]  # Хеш пустой строки
    for elem in string:
        # Новый хеш = (старый хеш * основание + код символа) mod простое число
        values.append(((values[-1] * base) + code(elem)) % mod)
    return values

# Параметры хеширования (лучше выбирать base и mod как взаимно простые числа)
base = 91          # Основание системы счисления
mod = 1000000321   # Большое простое число для уменьшения коллизий
# Ввод данных
target = input().strip()  # Подстрока для поиска (удаляем лишние пробелы)
text = input().strip()    # Текст, в котором ищем (удаляем лишние пробелы)
# Проверка на случай, когда искомая строка длиннее текста
if len(target) > len(text):
    print(-1)
    exit()
# Вычисление хешей
hash_target = poly_hash(target)[-1]  # Хеш всей искомой подстроки
hash_text = poly_hash(text)         # Хеши всех префиксов текста
index = []  # Список для хранения позиций вхождений
# Предвычисляем base^len_target mod mod для оптимизации
power = pow(base, len(target), mod)  # Эквивалентно (base**len_target) % mod, но эффективнее
# Основной цикл поиска
for i in range(len(text) - len(target) + 1):
    # Вычисляем хеш текущей подстроки text[i..i+len_target-1]
    current_hash = (hash_text[i + len(target)] - hash_text[i] * power) % mod
    # Если хеши совпали, возможно, мы нашли вхождение
    if hash_target == current_hash:
        # Дополнительная проверка на случай коллизии (можно добавить посимвольное сравнение)
        if text[i:i+len(target)] == target:
            index.append(str(i))  # Сохраняем индекс как строку для вывода

if index:
    print(" ".join(index))  # Выводим индексы через пробел
else:
    print(-1)  # Если вхождений не найдено

# 2. Открытая хеш и закрытая хеш-таблицы. Проблема удаления из закрытой хеш-таблицы. Перехеширование.

**Хеш-таблица** — это структура данных, которая использует хеш-функцию для преобразования ключей в индексы массива, что позволяет эффективно хранить и извлекать данные. Хеш-таблицы могут быть реализованы двумя основными способами: открытой и закрытой адресацией.

#### Открытая хеш-таблица

В открытой хеш-таблице все элементы хранятся в одном массиве, и при коллизии (когда два ключа хешируются в один и тот же индекс) используется метод разрешения коллизий. Основные методы разрешения коллизий в открытой хеш-таблице:

1. **Линейное пробирование**: Если возникает коллизия, мы просто проверяем следующий индекс в массиве, пока не найдем пустую ячейку.
2. **Квадратичное пробирование**: Вместо того чтобы проверять следующий индекс, мы проверяем индексы, увеличивая шаг в квадрате (например, 1, 4, 9 и т.д.).
3. **Двойное хеширование**: Используется вторая хеш-функция для определения шага при коллизии.

Преимущества открытой хеш-таблицы:
- Простота реализации.
- Эффективное использование памяти, так как все элементы хранятся в одном массиве.

Недостатки:
- При высокой нагрузке (большом количестве коллизий) производительность может значительно ухудшиться.

#### Закрытая хеш-таблица

В закрытой хеш-таблице элементы хранятся в отдельной структуре данных (например, связанных списках или деревьях) для каждого индекса массива. При коллизии элементы с одинаковым хеш-значением добавляются в эту структуру.

Преимущества закрытой хеш-таблицы:
- Более высокая производительность при большом количестве коллизий, так как элементы не перемещаются по массиву.
- Легче управлять удалением элементов, так как они могут быть просто удалены из связанного списка или дерева.

Недостатки:
- Использует больше памяти, так как для каждого индекса может потребоваться дополнительная структура данных.

### Проблема удаления из закрытой хеш-таблицы

Удаление элементов из закрытой хеш-таблицы может быть проблематичным, особенно если используется связанный список для разрешения коллизий. Основные проблемы, связанные с удалением:

1. **Потеря информации о коллизиях**: Если элемент удаляется, и он был единственным в своем списке, то это не создает проблем. Однако, если в списке есть другие элементы, то удаление может привести к тому, что некоторые из них станут недоступными, если не будет правильно обновлен указатель на следующий элемент.

2. **Необходимость поддержания структуры**: После удаления элемента необходимо убедиться, что структура данных (например, связанный список) остается корректной. Это может потребовать дополнительных операций, что увеличивает сложность.

3. **Проблемы с производительностью**: Если удаление происходит часто, это может привести к фрагментации памяти и ухудшению производительности, так как структура данных может стать неэффективной.

### Перехеширование

**Перехеширование** — это процесс, который используется для изменения размера хеш-таблицы и перераспределения элементов, когда таблица становится слишком заполненной или слишком пустой. Это может происходить по следующим причинам:

1. **Увеличение размера**: Когда количество элементов в хеш-таблице превышает определенный порог (например, 70% от общего размера), может потребоваться увеличение размера таблицы. Это делается для уменьшения вероятности коллизий и улучшения производительности.

2. **Уменьшение размера**: Если хеш-таблица становится слишком пустой (например, менее 30% заполненности), может быть целесообразно уменьшить ее размер, чтобы сэкономить память.

Процесс перехеширования включает следующие шаги:
1. Создание новой, более крупной или меньшей хеш-таблицы.
2. Пересчет хеш-значений для всех существующих элементов.
3. Вставка элементов в новую таблицу.

Перехеширование может быть затратным по времени, так как требует повторного вычисления хеш-значений и перемещения элементов. Однако оно необходимо для поддержания эффективности хеш-таблицы в долгосрочной перспективе.

In [ ]:
def polhash(data:str, a:int, b:int)->int:
    res = 0
    for x in data:
        res = (res * a + ord(x)) % b
    return res

def insert(table, key, value):
    hash = polhash(key, 91, 100)
    index = hash % 10
    for item in table[index]:
        if item[1] == key:
            item[2] = value  
            return
    table[index].append([hash, key, value])

def search(table, key:str):
    hash = polhash(key, 91, 100)
    index = hash % 10
    for i in table[index]:
            if i[0] == hash and i[1] == key:
                return i[2] 
    return 'KeyError'

def remove(table, key:str):
    hash = polhash(key, 91, 100)
    index = hash % 10
    for i in table[index]:
            if i[0] == hash and i[1] == key:
                table[index].remove(i)
                return i[2]
    return 'KeyError'

hash_table = [[] for _ in range(10)]
M = int(input())
for _ in range(M):
    key, value = map(str, input().split())
    insert(hash_table, key, value)

for i in range(len(hash_table)):
    if hash_table[i] == []:
        continue
    else:
        print(i)
        for j in range(len(hash_table[i])):
            print(*hash_table[i][j])

# 3. Словари и множества в Python.
Словари и множества — это две важные структуры данных в Python, которые позволяют эффективно хранить и обрабатывать данные.

### Словари (dict)

**Словарь** — это неупорядоченная коллекция пар "ключ-значение". Каждый ключ в словаре должен быть уникальным, и он используется для доступа к соответствующему значению. Словари реализованы с использованием хеш-таблиц, что обеспечивает быструю вставку, удаление и поиск элементов.

#### Основные свойства словарей:
- Ключи должны быть неизменяемыми (например, строки, числа, кортежи).
- Значения могут быть любого типа и могут дублироваться.

### Множества (set)

**Множество** — это неупорядоченная коллекция уникальных элементов. Множества также реализованы с использованием хеш-таблиц, что обеспечивает быструю проверку на наличие элемента, добавление и удаление.

#### Основные свойства множеств:
- Неупорядоченные: элементы не имеют фиксированного порядка.
- Все элементы должны быть уникальными.
- Множества могут содержать только неизменяемые (хешируемые) типы данных.

In [ ]:
# Создание словаря
person = {
    "name": "Alice",
    "age": 30,
    "city": "New York"
}

# Доступ к элементам
print(person["name"])  # Вывод: Alice
print(person.get("age"))  # Вывод: 30

# Изменение значения
person["age"] = 31

# Добавление нового элемента
person["job"] = "Engineer"

# Удаление элемента
del person["city"]

# Перебор ключей и значений
for key, value in person.items():
    print(f"{key}: {value}")

# Создание множества
fruits = {"apple", "banana", "cherry"}

# Добавление элемента
fruits.add("orange")

# Удаление элемента
fruits.remove("banana")

# Проверка на наличие элемента
if "apple" in fruits:
    print("Apple is in the set")

# Перебор элементов
for fruit in fruits:
    print(fruit)

# 4. Списки: односвязный, двусвязный, кольцо. Время работы основных операций (добавление в начало/конец, удаление с начала/конца, обращение к произвольному элементу).

Списки — это структуры данных, которые позволяют хранить коллекции элементов. В зависимости от реализации, списки могут иметь разные свойства и временные характеристики для выполнения основных операций. Рассмотрим три типа списков: односвязный, двусвязный и кольцевой.

### 1. Односвязный список

**Односвязный список** — это структура данных, в которой каждый элемент (узел) содержит данные и ссылку на следующий узел. Последний узел указывает на `None`, что обозначает конец списка.

#### Время работы основных операций:
- **Добавление в начало**: O(1) — просто создаем новый узел и перенаправляем его на текущую голову списка.
- **Добавление в конец**: O(n) — необходимо пройти весь список, чтобы найти последний узел.
- **Удаление с начала**: O(1) — просто перенаправляем голову на следующий узел.
- **Удаление с конца**: O(n) — необходимо пройти весь список, чтобы найти предпоследний узел.
- **Обращение к произвольному элементу**: O(n) — необходимо пройти список до нужного элемента.

### 2. Двусвязный список

**Двусвязный список** — это структура данных, в которой каждый узел содержит данные, ссылку на следующий узел и ссылку на предыдущий узел. Это позволяет легко перемещаться в обоих направлениях.

#### Время работы основных операций:
- **Добавление в начало**: O(1) — создаем новый узел и перенаправляем ссылки.
- **Добавление в конец**: O(1) — если у нас есть ссылка на последний узел, мы можем добавить новый узел сразу.
- **Удаление с начала**: O(1) — просто перенаправляем голову на следующий узел.
- **Удаление с конца**: O(1) — если у нас есть ссылка на последний узел, мы можем удалить его сразу.
- **Обращение к произвольному элементу**: O(n) — необходимо пройти список до нужного элемента.

### 3. Кольцевой список

**Кольцевой список** — это структура данных, в которой последний узел указывает на первый узел, образуя кольцо. Кольцевые списки могут быть как односвязными, так и двусвязными.

#### Время работы основных операций:
- **Добавление в начало**: O(1) — создаем новый узел

In [ ]:
class Node:
    def __init__(self, data, next=None):
        self.data = data    # Храним данные в узле
        self.next = next    # Ссылка на следующий узел (по умолчанию None)

    # Метод для вставки нового узла после текущего
    def insert(self, data):
        n = Node(data, self.next)  # Создаем новый узел, который ссылается на следующий узел текущего
        self.next = n              # Теперь текущий узел ссылается на новый

    # Метод для удаления следующего узла
    def remove(self):
        r = self.next.data          # получаем данные следующего узла
        self.next = self.next.next  # "Пропускаем" следующий узел, ссылаясь через один
        return r                    # Возвращаем данные удаленного узла


class LinkedList:
    def __init__(self):
        self.head = None  # Начало списка
        self.tail = None  # Конец списка 

    # Добавление элемента в начало списка
    def add(self, data):
        n = Node(data, self.head)  # Создаем новый узел, который ссылается на текущий head
        if self.head is None:      # Если список пуст
            self.head = self.tail = n  # Новый узел становится и head, и tail
        else:
            self.head = n          # Новый узел становится новым head

    def append(self, data):
        # Создаем новый узел, который ссылается на None
        n = Node(data)
        if self.head is None:
            # Если список пуст, новый узел становится head и tail
            self.head = self.tail = n
        else:
            # Если список не пуст, добавляем новый узел в конец
            self.tail.next = n  # Текущий tail теперь ссылается на новый узел
            self.tail = n       # Обновляем tail на новый узел

    # Удаление и возврат первого элемента списка
    def pop(self):
        if self.head is None:      # Если список пуст
            raise ValueError('pop from empty Linked List')
        res = self.head.data       # Сохраняем данные первого узла
        self.head = self.head.next  # Перемещаем head на следующий узел
        if self.head is None:      # Если список стал пустым
            self.tail = None       # Обнуляем tail
        return res                 # Возвращаем данные удаленного узла
    
    # Проверка, пуст ли список
    def is_empty(self):
        return self.head is None  # True, если head равен None

if __name__ == "__main__":
    from random import randint
    l = LinkedList()  # Создаем пустой связный список
    # Добавляем 10 случайных чисел в список
    for _ in range(10):
        x = randint(10, 99)  # Генерируем случайное число
        print(x, end=' ')    # Печатаем его
        l.append(x)             # Добавляем в начало списка
    print()
    # Извлекаем и печатаем все элементы, пока список не станет пустым
    while not l.is_empty():
        print(l.pop(), end=' ')  # Печатаем элементы в порядке извлечения
    print()  # Печатаем пустую строку для красоты

# 5. Двоичное дерево поиска (Binary Search Tree, BST)

**Двоичное дерево поиска (BST)** — это структура данных, в которой каждый узел имеет не более двух дочерних узлов (левый и правый). Для любого узла:
- Все элементы в **левом поддереве** меньше его значения.
- Все элементы в **правом поддереве** больше его значения.

### **Свойства BST:**
1. **Уникальность ключей** (обычно, но иногда допускаются дубликаты).
2. **Быстрый поиск** (в среднем за `O(log n)`, но в худшем случае `O(n)` при несбалансированном дереве).
3. **Упорядоченность** (обход **in-order** даёт отсортированную последовательность).

In [ ]:
class Node:
    def __init__(self, key):
        self.key = key      # Значение ключа узла
        self.left = None    # Ссылка на левого потомка
        self.right = None   # Ссылка на правого потомка

class BinarySearchTree:
    def __init__(self):
        self.root = None  # Корень дерева

    def insert(self, key):
        """Вставка нового узла."""
        if self.root is None:
            self.root = Node(key)  # Если дерево пустое, создаем корень
        else:
            self._insert_recursive(self.root, key)  # Иначе вызываем рекурсивную вставку

    def _insert_recursive(self, node, key):
        """Рекурсивная вставка."""
        if key < node.key:
            if node.left is None:
                node.left = Node(key)  # Если левый потомок отсутствует, добавляем новый узел
            else:
                self._insert_recursive(node.left, key)  # Иначе продолжаем поиск в левом поддереве
        elif key > node.key:
            if node.right is None:
                node.right = Node(key)  # Если правый потомок отсутствует, добавляем новый узел
            else:
                self._insert_recursive(node.right, key)  # Иначе продолжаем поиск в правом поддереве
        # Если ключ уже существует, ничего не делаем (можно обработать иначе)

    def search(self, key):
        """Поиск узла по ключу."""
        return self._search_recursive(self.root, key)

    def _search_recursive(self, node, key):
        """Рекурсивный поиск."""
        if node is None or node.key == key:
            return node  # Если узел найден или достигнут конец дерева, возвращаем узел
        if key < node.key:
            return self._search_recursive(node.left, key)  # Продолжаем поиск в левом поддереве
        return self._search_recursive(node.right, key)  # Продолжаем поиск в правом поддереве

    def delete(self, key):
        """Удаление узла."""
        self.root = self._delete_recursive(self.root, key)

    def _delete_recursive(self, node, key):
        """Рекурсивное удаление."""
        if node is None:
            return node  # Если узел не найден, возвращаем None
        if key < node.key:
            node.left = self._delete_recursive(node.left, key)  # Ищем в левом поддереве
        elif key > node.key:
            node.right = self._delete_recursive(node.right, key)  # Ищем в правом поддереве
        else:
            # Узел с одним или без детей
            if node.left is None:
                return node.right  # Если нет левого потомка, возвращаем правого
            elif node.right is None:
                return node.left  # Если нет правого потомка, возвращаем левого
            # Узел с двумя детьми: находим минимальный в правом поддереве
            temp = self._find_min(node.right)
            node.key = temp.key  # Заменяем ключ текущего узла на минимальный
            node.right = self._delete_recursive(node.right, temp.key)  # Удаляем минимальный узел
        return node

    def _find_min(self, node):
        """Находит минимальный узел в поддереве."""
        current = node
        while current.left is not None:
            current = current.left
        return current

    def inorder_traversal(self):
        """Обход дерева в порядке in-order (сортировка по возрастанию)."""
        result = []
        self._inorder_recursive(self.root, result)
        return result

    def _inorder_recursive(self, node, result):
        if node:
            self._inorder_recursive(node.left, result)
            result.append(node.key)
            self._inorder_recursive(node.right, result)

# 6. Куча (Heap)

**Куча** — это специальная структура данных, которая представляет собой полное двоичное дерево, удовлетворяющее свойству кучи. Существует два основных типа кучи:

1. **Макс-куча (Max-Heap)**: В этой структуре для каждого узла выполняется условие, что значение узла больше или равно значениям его дочерних узлов. Таким образом, корень дерева содержит максимальное значение.

2. **Мин-куча (Min-Heap)**: В этой структуре для каждого узла выполняется условие, что значение узла меньше или равно значениям его дочерних узлов. Корень дерева содержит минимальное значение.

### Основные операции с кучей

Куча поддерживает несколько основных операций:

1. **Вставка (Insert)**: Добавление нового элемента в кучу. Элемент добавляется в конец дерева, а затем происходит "всплытие" (sift-up) для восстановления свойства кучи.

2. **Удаление корня (Extract)**: Удаление корневого элемента (максимального или минимального). Корень заменяется последним элементом в куче, и затем происходит "погружение" (sift-down) для восстановления свойства кучи.

3. **Построение кучи (Heapify)**: Преобразование массива в кучу. Это можно сделать за O(n) времени.

4. **Просмотр корня (Peek)**: Получение значения корня без его удаления.

### Время работы операций

- **Вставка**: O(log n)
- **Удаление корня**: O(log n)
- **Построение кучи**: O(n)
- **Просмотр корня**: O(1)

### Сортировка кучей (Heap Sort)

**Сортировка кучей** — это алгоритм сортировки, который использует структуру данных "куча". Он работает следующим образом:

1. **Построение кучи**: Преобразуем массив в кучу (макс-кучу для сортировки по возрастанию).

2. **Удаление корня**: Извлекаем корень (максимальный элемент) и помещаем его в конец массива. Затем уменьшаем размер кучи и восстанавливаем ее свойства.

3. **Повторение**: Повторяем шаг 2, пока не отсортируем весь массив.

### Принципы работы сортировки кучей

1. **Эффективность**: Сортировка кучей имеет временную сложность O(n log n) в худшем, среднем и лучшем случаях, что делает ее эффективной для сортировки больших массивов.

2. **Не требует дополнительной памяти**: Сортировка кучей выполняется на месте, что означает, что она не требует дополнительной памяти для хранения временных массивов, как это делает, например, сортировка слиянием.

In [ ]:
class Heap:
    def __init__(self):
        self.__h = []  # Список для хранения элементов кучи

    def push(self, el):
        """
        Добавляет элемент в кучу.
        После добавления восстанавливает свойства кучи с помощью shift_up.
        """
        self.__h.append(el)  # Добавляем элемент в конец списка
        self.__shift_up(len(self.__h) - 1)  # Восстанавливаем свойства кучи

    def __shift_up(self, n):
        """
        Поднимает элемент вверх по дереву, чтобы восстановить свойства кучи.
        :param n: Индекс элемента, который нужно поднять.
        """
        if n == 0:  # Если достигли корня, останавливаемся
            return
        p = (n - 1) // 2  # Индекс родительского элемента
        if self.__h[p] > self.__h[n]:  # Если родитель больше текущего элемента
            self.__h[p], self.__h[n] = self.__h[n], self.__h[p]  # Меняем их местами
            self.__shift_up(p)  # Рекурсивно продолжаем подъем

    def pop(self):
        """
        Удаляет минимальный элемент из кучи (корень).
        После удаления восстанавливает свойства кучи с помощью shift_down.
        """
        if not self.__h:  # Если куча пуста, выбрасываем исключение
            raise IndexError("Heap is empty")
        res = self.__h[0]  # Запоминаем минимальный элемент (корень)
        self.__h[0] = self.__h[-1]  # Перемещаем последний элемент на место корня
        self.__h.pop()  # Удаляем последний элемент
        if len(self.__h):  # Если куча не пуста, восстанавливаем свойства
            self.__shift_down(0)
        return res  # Возвращаем минимальный элемент

    def __shift_down(self, i):
        """
        Опускает элемент вниз по дереву, чтобы восстановить свойства кучи.
        :param i: Индекс элемента, который нужно опустить.
        """
        size = len(self.__h)  # Размер кучи
        while True:
            left = 2 * i + 1  # Левый потомок
            right = 2 * i + 2  # Правый потомок
            smallest = i  # Индекс наименьшего элемента
            # Проверяем, есть ли левый потомок и является ли он меньше текущего
            if left < size and self.__h[left] < self.__h[smallest]:
                smallest = left
            # Проверяем, есть ли правый потомок и является ли он меньше текущего
            if right < size and self.__h[right] < self.__h[smallest]:
                smallest = right
            if smallest == i:  # Если текущий элемент уже на своем месте
                break
            # Меняем текущий элемент с наименьшим из потомков
            self.__h[i], self.__h[smallest] = self.__h[smallest], self.__h[i]
            i = smallest  # Продолжаем с новой позиции

    @staticmethod
    def heapify(a):
        """
        Преобразует список в кучу (heapify).
        :param a: Список, который нужно преобразовать в кучу.
        :return: Новый объект Heap, содержащий элементы списка.
        """
        h = Heap()
        h.__h = a[:]  # Копируем список, чтобы не изменять оригинальный
        for i in range((len(a) - 1) // 2, -1, -1):  # Проходим по всем узлам, начиная с последнего родителя
            h.__shift_down(i)
        return h

    @staticmethod
    def sort(a):
        """
        Сортирует список с использованием кучи (heap sort).
        :param a: Список, который нужно отсортировать.
        """
        h = Heap()
        h.__h = a  # Работаем с оригинальным списком
        for i in range((len(a) - 1) // 2, -1, -1):  # Преобразуем список в кучу
            h.__shift_down(i)

        for i in range(len(a) - 1, 0, -1):  # Извлекаем элементы из кучи
            h.__h[0], h.__h[i] = h.__h[i], h.__h[0]  # Меняем первый и последний элементы
            h.__shift_down(0, i)  # Восстанавливаем свойства кучи для уменьшенной части списка

    def __shift_down(self, i, maxN=None):
        """
        Перегруженная версия метода shift_down для использования в heap sort.
        :param i: Индекс элемента, который нужно опустить.
        :param maxN: Максимальный размер рассматриваемой части кучи.
        """
        size = maxN if maxN is not None else len(self.__h)  # Размер кучи
        while True:
            left = 2 * i + 1  # Левый потомок
            right = 2 * i + 2  # Правый потомок
            smallest = i  # Индекс наименьшего элемента

            if left < size and self.__h[left] < self.__h[smallest]:
                smallest = left

            if right < size and self.__h[right] < self.__h[smallest]:
                smallest = right

            if smallest == i:  # Если текущий элемент уже на своем месте
                break

            self.__h[i], self.__h[smallest] = self.__h[smallest], self.__h[i]
            i = smallest


if __name__ == "__main__":
    from random import randint

    # Тестирование push и pop
    h = Heap()
    for _ in range(10):
        x = randint(10, 99)
        print(x, end=' ')
        h.push(x)
    print()
    print(h._Heap__h)  # Вывод внутреннего состояния кучи
    for _ in range(10):
        print(h.pop(), end=' ')  # Извлечение элементов из кучи
    print()
    # Тестирование heapify и sort
    a = [randint(10, 99) for _ in range(10)]
    print("Исходный список:", a)
    b = Heap.heapify(a)
    print("Куча после heapify:", b._Heap__h)
    Heap.sort(a)
    print("Отсортированный список:", a)

93 93 41 16 91 59 62 48 74 94 
[16, 41, 59, 48, 91, 93, 62, 93, 74, 94]
16 41 48 59 62 74 91 93 93 94 
Исходный список: [40, 65, 29, 64, 25, 47, 70, 57, 19, 58]
Куча после heapify: [19, 25, 29, 57, 40, 47, 70, 65, 64, 58]
Отсортированный список: [70, 65, 64, 58, 57, 47, 40, 29, 25, 19]


# 7. Определение графа. Степень вершины, петли, кратные рёбра. Цепи, пути и циклы.

**Граф** — это математическая структура, состоящая из:
- **Вершин (узлов)** (обозначаются обычно `V` или `U, V, ...`)
- **Рёбер (дуг)** (обозначаются `E`), которые соединяют вершины.

### **Виды графов:**
- **Неориентированный граф** — рёбра не имеют направления (`(A, B)` = `(B, A)`).
- **Ориентированный граф (орграф)** — рёбра имеют направление (`A → B` ≠ `B → A`).
- **Взвешенный граф** — каждому ребру присвоен вес (число).
- **Не взвешенный граф** — рёбра не имеют весов.

## **Степень вершины**
- **В неориентированном графе**:
  - **Степень вершины** (`deg(v)`) — количество рёбер, инцидентных вершине.
  - **Петля** (ребро `(v, v)`) увеличивает степень вершины на **2**.

- **В ориентированном графе**:
  - **Полустепень исхода (`out-degree`)** — число выходящих рёбер.
  - **Полустепень захода (`in-degree`)** — число входящих рёбер.

## **Петли и кратные рёбра**
- **Петля** — ребро, соединяющее вершину саму с собой (`(A, A)`). 

- **Кратные рёбра** — несколько рёбер между одной парой вершин.   

- **Простой граф** — граф **без петель и кратных рёбер**.
- **Мультиграф** — граф **с кратными рёбрами и/или петлями**.

# 8. Сильная и слабая связность графа. Компоненты связности.

### **Неориентированные графы**
- **Связный граф** — если между **любыми двумя вершинами** существует путь.
- **Компонента связности** — максимальный связный подграф (все вершины достижимы друг из друга).

**Пример**:  
```
A — B    C — D  
 \ /     |  
  E     F — G
```
- **Компоненты связности**:  
  - `{A, B, E}`  
  - `{C, D, F, G}`  

### **Ориентированные графы (орграфы)**
Здесь понятие связности сложнее из-за направленности рёбер.

#### **Сильная связность**
- Орграф **сильно связный**, если для **любых двух вершин `u` и `v`** существует:  
  - путь из `u` в `v`,  
  - и путь из `v` в `u`.  
- **Компонента сильной связности (КСС)** — максимальный сильно связный подграф.

**Пример**:  
```
A → B → C  
↑   ↓   ↓  
D ← E ← F
```
- **КСС**:  
  - `{A, B, E, D}` (можно добраться из любой вершины в любую).  
  - `{C}` и `{F}` — отдельные КСС, так как из `C` нельзя вернуться.  

#### **Слабая связность**
- Орграф **слабо связный**, если при **игнорировании направлений рёбер** он становится связным неориентированным графом.
- **Компонента слабой связности** — аналогично компоненте в неориентированном графе.

**Пример**:  
```
A → B ← C  
 \   /  
   D
```
- **Слабо связный** (если сделать рёбра неориентированными, все вершины связаны).  
- **Не сильно связный** (из `C` нельзя попасть в `A`).  

## **Алгоритмы поиска компонент связности**
### **Для неориентированных графов**  
Используется **DFS (Depth-First Search)** или **BFS (Breadth-First Search)**.  

**Алгоритм**:  
1. Выбрать непосещённую вершину.  
2. Запустить DFS/BFS, пометив все достижимые вершины.  
3. Эти вершины образуют компоненту связности.  
4. Повторять, пока есть непосещённые вершины.  

**Алгоритм Косарайю**:  
1. **Первый обход DFS** — запоминаем порядок выхода.  
2. **Транспонируем граф** (разворачиваем рёбра).  
3. **Второй обход DFS** в порядке убывания времени выхода.  

In [ ]:
def dfs(graph, v, visited):
    visited[v] = True
    for neighbor in graph[v]:
        if not visited[neighbor]:
            dfs(graph, neighbor, visited)

def is_strongly_connected(N, edges):
    # Создаем граф и его транспонированный граф
    graph = {i: [] for i in range(N)}
    transposed_graph = {i: [] for i in range(N)}
    
    for u, v in edges:
        graph[u].append(v)
        transposed_graph[v].append(u)
    
    # Шаг 1: Выполняем DFS из первой вершины (например, 0)
    visited = [False] * N
    dfs(graph, 0, visited)
    
    # Если не все вершины посещены, граф не сильно связный
    if any(not v for v in visited):
        return False
    
    # Шаг 2: Обнуляем массив посещений
    visited = [False] * N
    
    # Шаг 3: Выполняем DFS из той же вершины в транспонированном графе
    dfs(transposed_graph, 0, visited)
    
    # Если не все вершины посещены, граф не сильно связный
    if any(not v for v in visited):
        return False
    
    # Если все вершины посещены в обоих графах, граф сильно связный
    return True

N = int(input())
M = int(input())
edges = []
for _ in range(M):
    u, v = map(int, input().split())
    edges.append((u, v))

if is_strongly_connected(N, edges): print("YES")
else: print("NO")

# 9. Способы представления графа в памяти.

In [ ]:
# матрица смежности 
def read_matrix():
    # Считываем количество вершин (N) и количество рёбер (M)
    N, M = map(int, input().split())
    # Создаем матрицу смежности размером N x N, заполненную нулями
    G = [[0] * N for _ in range(N)]
    # Массив для хранения имен вершин
    name = []
    # Считываем рёбра
    for _ in range(M):
        v, u = input().split()  # Считываем две вершины ребра
        # Если вершина v ещё не добавлена в массив имен, добавляем её
        if v not in name:
            name.append(v)
        # Если вершина u ещё не добавлена в массив имен, добавляем её
        if u not in name:
            name.append(u)
        # Получаем индексы вершин v и u в массиве имен
        v = name.index(v)
        u = name.index(u)
        # Устанавливаем ребро от u к v (убираем, если граф ориентированный)
        G[u][v] = 1
        # Устанавливаем ребро от v к u (убираем, если граф ориентированный)
        G[v][u] = 1
    # Возвращаем матрицу смежности и массив имен вершин
    return G, name

# список смежности
def read_list(): 
    N, M = map(int, input().split())
    G = [[] * N for _ in range(N)]
    name = []
    for _ in range(M):
        v, u = input().split()
        if v not in name: name.append(v)
        if u not in name: name.append(u)
        v = name.index(v)
        u = name.index(u)
        G[u].append(v) # убрать, если граф ориентированный
        G[v].append(u) 
    return G, name

# медленный (не для кр)
# Функция для чтения графа в виде словаря смежности
def read_graph(orient=False, weight=False):
    # Считываем количество вершин (N) и количество рёбер (M)
    N, M = map(int, input().split())
    # Создаем словарь смежности
    G = {}
    # Считываем рёбра
    for _ in range(M):
        if not weight:
            u, v = input().split()  # Считываем две вершины ребра
            # Если вершина u ещё не добавлена в словарь, добавляем её с пустым списком соседей
            if u not in G:
                G[u] = []
            # Если вершина v ещё не добавлена в словарь, добавляем её с пустым списком соседей
            if v not in G:
                G[v] = []
            # Добавляем вершину v в список соседей вершины u (убираем, если граф ориентированный)
            G[u].append(v)
            # Добавляем вершину u в список соседей вершины v (убираем, если граф ориентированный)
            if not orient:
                G[v].append(u)
        else:
            u, v, w = input().split()  # Считываем две вершины ребра и вес
            w = float(w)  # Преобразуем вес в число с плавающей точкой
            # Если вершина u ещё не добавлена в словарь, добавляем её с пустым словарём соседей
            if u not in G:
                G[u] = {}
            # Если вершина v ещё не добавлена в словарь, добавляем её с пустым словарём соседей
            if v not in G:
                G[v] = {}
            # Устанавливаем ребро от u к v с весом w (убираем, если граф ориентированный)
            G[u][v] = w
            # Устанавливаем ребро от v к u с весом w (убираем, если граф ориентированный)
            if not orient:
                G[v][u] = w
    # Возвращаем словарь смежности
    return G

# компактный способ хранения графа
# реализация без петель 
def compact(G:list[list[int]]):
    offset = [0]
    edges = []
    for lst in G:
        for ends in lst:
            edges.append(ends)
        offset.append(len(edges))
    return offset, edges

if __name__ == "__main__":
    #1
    G, name = read_matrix()
    print(" ", *name)
    for i in range(len(name)):
        print(name[i], *G[i], sum(G[i]))

    #2
    G, name = read_list()
    for i in range(len(name)):
        print(name[i], ':', *map(lambda x:name[x], G[i]), len(G[i]))

    #3
    G = read_graph()
    for v in G:
        print(v, ":", *G[v], len(G[v]))

# 10. Выделение компоненты связности обходом в глубину.

**Обход в глубину** — это алгоритм, который исследует граф, двигаясь **как можно глубже** по одной ветви перед возвратом.  

Для **выделения компоненты связности**:
1. Запускаем DFS из **произвольной непосещённой вершины**.
2. Все вершины, достижимые из неё, образуют **одну компоненту связности**.
3. Повторяем для оставшихся непосещённых вершин.

## **Алгоритм**

### **Для ориентированного графа (КСС)**
Используется **алгоритм Косарайю** (два прохода DFS):
1. Первый DFS — определяем порядок выхода.
2. Транспонируем граф.
3. Второй DFS — выделяем КСС.

## **Сложность**
- **Время**: `O(V + E)` (каждый узел и ребро посещаются один раз).
- **Память**: `O(V)` (хранение посещённых вершин).

In [ ]:
def dfs(G, v, visited, component):
    # Отмечаем текущую вершину как посещенную
    visited[v] = True
    # Добавляем текущую вершину в компоненту связности
    component.append(v)
    # Перебираем всех соседей текущей вершины
    for neighbor in G[v]:
        # Если соседняя вершина не была посещена, рекурсивно вызываем DFS
        if not visited[neighbor]:
            dfs(G, neighbor, visited, component)

def find_connected_components(G):
    # Список для хранения всех компонент связности
    components = []
    # Список посещенных вершин, изначально все вершины не посещены
    visited = {v: False for v in G}
    # Перебираем все вершины графа
    for v in G:
        # Если вершина не была посещена, запускаем DFS
        if not visited[v]:
            # Создаем список для текущей компоненты связности
            component = []
            # Запускаем DFS для вершины v
            dfs(G, v, visited, component)
            # Добавляем найденную компоненту связности в список всех компонент
            components.append(component)
    return components

# Функция для чтения графа в виде списка смежности
def read_graph():
    # Считываем количество вершин (N) и количество рёбер (M)
    N, M = map(int, input().split())
    # Создаем словарь смежности
    G = {}
    # Считываем рёбра
    for _ in range(M):
        u, v = input().split()  # Считываем две вершины ребра
        # Если вершина u ещё не добавлена в словарь, добавляем её с пустым списком соседей
        if u not in G:
            G[u] = []
        # Если вершина v ещё не добавлена в словарь, добавляем её с пустым списком соседей
        if v not in G:
            G[v] = []
        # Добавляем вершину v в список соседей вершины u
        G[u].append(v)
        # Добавляем вершину u в список соседей вершины v
        G[v].append(u)
    # Возвращаем словарь смежности
    return G

G = read_graph()
components = find_connected_components(G)
for i, component in enumerate(components):
    print(f"Компонента связности {i + 1}: {component}")

# 11. Проверка двудольности графа.

**Двудольный граф (Bipartite Graph)** — это граф, вершины которого можно разделить на **два непересекающихся множества** `U` и `V` так, что:
- Каждое ребро соединяет вершину из `U` с вершиной из `V`.
- **Нет рёбер** между вершинами внутри одного множества.

## **Критерии двудольности**
Граф **двудольный** тогда и только тогда, когда он:
- **Не содержит циклов нечётной длины** (т.е. все циклы чётные).
- **Может быть раскрашен в 2 цвета** (например, красный и синий) так, что соседние вершины имеют разные цвета.

## **Алгоритм проверки двудольности**
Используется **модифицированный BFS или DFS** с раскраской вершин.

1. Выбрать стартовую вершину, покрасить её в цвет `0`.
2. Для каждой вершины `u`:
   - Все её соседи `v` должны быть покрашены в цвет `1 - color[u]`.
   - Если сосед уже раскрашен и его цвет совпадает с `u` → граф **не двудольный**.
3. Если все вершины раскрашены без конфликтов → граф **двудольный**.

## **Сложность**
- **Время**: `O(V + E)` (как BFS/DFS).
- **Память**: `O(V)` (хранение цветов).

In [ ]:
def is_bipartite_dfs(graph):
    # Создаем словарь `color`, который будет хранить цвет каждой вершины.
    # Если вершина еще не окрашена, она отсутствует в словаре.
    color = {}
    # Проходим по всем вершинам графа. Это нужно для обработки несвязных компонент графа.
    for node in graph:
        # Если вершина еще не окрашена, начинаем DFS с этой вершины.
        if node not in color:
            # Инициализируем стек для DFS. Каждый элемент стека — это кортеж `(вершина, цвет)`.
            # Начинаем с текущей вершины `node` и присваиваем ей цвет 0.
            stack = [(node, 0)]
            color[node] = 0
            # Запускаем DFS с использованием стека.
            while stack:
                # Извлекаем вершину `u` и её цвет `c` из стека.
                u, c = stack.pop()
                # Обходим всех соседей вершины `u`.
                for v in graph[u]:
                    # Если соседняя вершина `v` еще не окрашена:
                    if v not in color:
                        # Окрашиваем её в противоположный цвет (1 - c).
                        color[v] = 1 - c
                        # Добавляем её в стек для дальнейшего обхода.
                        stack.append((v, color[v]))
                    # Если соседняя вершина `v` уже окрашена и её цвет совпадает с цветом `u`:
                    elif color[v] == c:
                        # Граф не является двудольным, так как две смежные вершины имеют одинаковый цвет.
                        return False
    # Если все вершины успешно окрашены без конфликтов, граф является двудольным.
    return True

# 12. Проверка графа на ацикличность или нахождение цикла обходом в глубину.

- **Ациклический граф** — граф, не содержащий циклов.
- **Цикл** — путь, у которого начальная и конечная вершины совпадают.

## **Алгоритм проверки на ацикличность с помощью DFS**
При обходе в глубину можно обнаружить цикл, если мы встречаем вершину, которая уже посещена и не является родительской.

1. Запускаем DFS из каждой непосещённой вершины.
2. Для каждой вершины храним её **родителя** (предыдущую вершину в обходе).
3. Если мы встречаем уже посещённую вершину, которая **не является родителем** текущей, значит, найден цикл.

## **Сложность**
- **Время**: `O(V + E)` (как у обычного DFS).
- **Память**: `O(V)` (хранение посещённых вершин и родителей).

In [ ]:
def has_cycle_dfs(graph):
    visited = set()
    def dfs(node, parent):
        visited.add(node)
        for neighbor in graph[node]:
            if neighbor not in visited:
                if dfs(neighbor, node):
                    return True
            elif neighbor != parent:
                return True
        return False
    
    for node in graph:
        if node not in visited:
            if dfs(node, None):
                return True
    return False

def has_cycle_directed_dfs(graph):
    visited = set()
    in_stack = set()
    def dfs(node):
        visited.add(node)
        in_stack.add(node)
        for neighbor in graph.get(node, []):
            if neighbor not in visited:
                if dfs(neighbor):
                    return True
            elif neighbor in in_stack:
                return True
        in_stack.remove(node)
        return False
    for node in graph:
        if node not in visited:
            if dfs(node):
                return True
    return False

def find_cycle_dfs(graph):
    visited = {}
    parent = {}
    cycle = []
    def dfs(node, par):
        visited[node] = True
        parent[node] = par
        for neighbor in graph[node]:
            if neighbor not in visited:
                if dfs(neighbor, node):
                    return True
            elif neighbor != par:
                # Найден цикл, восстанавливаем его
                current = node
                while current != neighbor:
                    cycle.append(current)
                    current = parent[current]
                cycle.append(neighbor)
                cycle.append(node)  # Замыкаем цикл
                return True
        return False
    for node in graph:
        if node not in visited:
            if dfs(node, None):
                return cycle[::-1]  # Разворачиваем для правильного порядка
    return None

# 13. Алгоритм Косарайю выделения компонент сильной связности орграфа.

В **ориентированном графе** (орграфе) компонента сильной связности — это максимальный подграф, в котором:
- **Любые две вершины достижимы друг из друга** (существует путь `u → v` и `v → u`).

## **Алгоритм Косарайю**
Алгоритм состоит из **двух проходов DFS**:
1. **Первый DFS** — определяет порядок выхода вершин (обратный топологический порядок).
2. **Транспонирование графа** — разворот всех рёбер.
3. **Второй DFS** — обрабатывает вершины в порядке первого шага, выделяя КСС.

### **Почему работает?**
- Транспонированный граф **сохраняет КСС**, но **меняет достижимость между ними**.
- Обработка вершин в порядке убывания времени выхода гарантирует, что мы сначала посещаем **"корневые" КСС**.

## **Сложность алгоритма**
- **Время**: `O(V + E)` (два прохода DFS + транспонирование).
- **Память**: `O(V + E)` (хранение графа и транспонированного).

In [ ]:
def kosaraju(graph):
    """:return: Список компонент сильной связности. """
    # Шаг 1: Создаем транспонированный граф (обратные рёбра)
    def transpose_graph(graph):
        transposed = {}
        for u in graph:
            for v in graph[u]:
                if v not in transposed:
                    transposed[v] = []  # Инициализируем пустой список
                transposed[v].append(u)
        return transposed

    # Шаг 2: DFS для получения порядка завершения обработки вершин
    def dfs_order(graph, node, visited, stack):
        visited.add(node)
        for neighbor in graph.get(node, []):  # Используем .get() для избежания KeyError
            if neighbor not in visited:
                dfs_order(graph, neighbor, visited, stack)
        stack.append(node)

    # Шаг 3: DFS для выделения компонент сильной связности
    def dfs_scc(graph, node, visited, scc):
        visited.add(node)
        scc.append(node)
        for neighbor in graph.get(node, []):  # Используем .get() для избежания KeyError
            if neighbor not in visited:
                dfs_scc(graph, neighbor, visited, scc)

    # Инициализация
    visited = set()
    stack = []
    scc_list = []

    # Шаг 4: Первый проход DFS на исходном графе для заполнения стека
    for node in graph:
        if node not in visited:
            dfs_order(graph, node, visited, stack)

    # Шаг 5: Создание транспонированного графа
    transposed_graph = transpose_graph(graph)

    # Шаг 6: Второй проход DFS на транспонированном графе
    visited.clear()
    while stack:
        node = stack.pop()
        if node not in visited:
            scc = []
            dfs_scc(transposed_graph, node, visited, scc)
            scc_list.append(scc)

    return scc_list

# Пример использования
# Ориентированный граф в виде списка смежности (обычный словарь)
graph = {
    0: [1],
    1: [2],
    2: [0, 3],
    3: [4],
    4: [5, 6],
    5: [3],
    6: [7],
    7: [8],
    8: [6, 9],
    9: []
}
# Вызов алгоритма Косарайю
scc_list = kosaraju(graph)
# Вывод результатов
print("Компоненты сильной связности:")
for i, scc in enumerate(scc_list, 1):
    print(f"SCC {i}: {scc}")

Компоненты сильной связности:
SCC 1: [0, 2, 1]
SCC 2: [3, 5, 4]
SCC 3: [6, 8, 7]
SCC 4: [9]


# 14. Выделение компонент связности обходом в ширину.

**Компонента связности** в неориентированном графе — это максимальный подграф, в котором:
- Любые две вершины соединены путём.
- Нет рёбер между вершинами из разных компонент.

## **Алгоритм выделения компонент связности BFS**
### **Шаги алгоритма**:
1. **Инициализация**:  
   - Создаём массив `visited` для отметки посещённых вершин.  
   - Создаём список `components` для хранения компонент связности.  

2. **Для каждой вершины**:  
   - Если вершина **не посещена**, запускаем из неё BFS.  
   - Все посещённые вершины в этом BFS образуют **новую компоненту связности**.  

3. **BFS для одной компоненты**:  
   - Используем очередь для обхода в ширину.  
   - Помечаем вершины как посещённые и добавляем их в текущую компоненту.  

In [ ]:
from collections import deque

def find_connected_components_bfs(graph):
    """:return: Список компонент связности."""
    # Множество для отслеживания посещённых вершин
    visited = set()
    # Список для хранения всех компонент связности
    components = []
    # Обходим все вершины графа
    for node in graph:
        # Если вершина ещё не посещена, начинаем BFS из этой вершины
        if node not in visited:
            # Очередь для BFS
            queue = deque([node])
            # Добавляем текущую вершину в множество посещённых
            visited.add(node)
            # Список для хранения текущей компоненты связности
            component = []
            # Начинаем обход в ширину
            while queue:
                # Извлекаем вершину из очереди
                current = queue.popleft() 
                # Добавляем её в текущую компоненту связности
                component.append(current)
                # Обходим всех соседей текущей вершины
                for neighbor in graph.get(current, []):  # Используем .get() для избежания KeyError
                    # Если соседняя вершина ещё не посещена
                    if neighbor not in visited:
                        # Добавляем её в множество посещённых
                        visited.add(neighbor)        
                        # Добавляем её в очередь для дальнейшего обхода
                        queue.append(neighbor)
            # После завершения BFS добавляем найденную компоненту в список компонент
            components.append(component)
    # Возвращаем список всех компонент связности
    return components

# Неориентированный граф в виде списка смежности (обычный словарь)
graph = {
    0: [1, 2],
    1: [0, 2],
    2: [0, 1],
    3: [4],
    4: [3],
    5: []
}

connected_components = find_connected_components_bfs(graph)
print("Компоненты связности:")
for i, component in enumerate(connected_components, 1):
    print(f"Компонента {i}: {component}")

Компоненты связности:
Компонента 1: [0, 1, 2]
Компонента 2: [3, 4]
Компонента 3: [5]


# 15. Нахождение кратчайшего цикла в невзвешенном графе.

In [ ]:
from collections import deque

# Ввод данных
n, m, x, y = map(int, input().split())  # Количество вершин, рёбер, начальная и конечная вершины
graph = [[] for _ in range(n)]          # Создаём список смежности
# Считываем рёбра и строим граф
for _ in range(m):
    u, v = map(int, input().split())
    graph[u].append(v)  # Добавляем ребро u -> v
    graph[v].append(u)  # Добавляем ребро v -> u (граф неориентированный)
# Обход в ширину (BFS)
def bfs(start, end):
    queue = deque([start])               # Очередь для BFS
    distance = [-1] * n                  # Массив расстояний, инициализируем -1 (не посещено)
    distance[start] = 0                  # Расстояние до стартовой вершины равно 0
    while queue:
        current = queue.popleft()         # Берём вершину из очереди
        # Если достигли целевой вершины, возвращаем расстояние
        if current == end:
            return distance[current]
        # Перебираем всех соседей текущей вершины
        for neighbor in graph[current]:
            if distance[neighbor] == -1:  # Если сосед ещё не посещён
                distance[neighbor] = distance[current] + 1  # Обновляем расстояние
                queue.append(neighbor)    # Добавляем соседа в очередь
    # Если конечная вершина недостижима, возвращаем -1
    return -1

result = bfs(x, y)
print(result)

# 16. Алгоритм Дейкстры (наивная реализация).

Время: O(V² + E) (каждый из V шагов требует поиска минимума за O(V)).

Память: O(V) (хранение расстояний и посещённых вершин).

In [ ]:
import sys
def dijkstra_naive(graph, start):
    # Инициализация
    n = len(graph)
    visited = [False] * n
    distance = [sys.maxsize] * n  # Изначально все расстояния "бесконечны"
    distance[start] = 0  # Расстояние до стартовой вершины = 0
    for _ in range(n):
        # Находим вершину с минимальным расстоянием среди непосещённых
        min_dist = sys.maxsize
        u = -1
        for v in range(n):
            if not visited[v] and distance[v] < min_dist:
                min_dist = distance[v]
                u = v
        if u == -1:  # Если все вершины посещены или недостижимы
            break
        visited[u] = True
        # Обновляем расстояния до соседей u
        for v, weight in graph[u]:
            if not visited[v] and distance[u] + weight < distance[v]:
                distance[v] = distance[u] + weight
    return distance

# 17. Алгоритм Дейкстры с кучей.

Работает только с неотрицательными весами (как и должен алгоритм Дейкстры)

Использует жадную стратегию - всегда обрабатывает ближайшую доступную вершину

Сложность: O(E + V log V) благодаря использованию кучи

In [ ]:
from random import randint

def read_graph():
    """Чтение графа из входных данных и создание его представления в виде словаря смежности"""
    G = {}  # Создаем пустой словарь для хранения графа
    n, m = map(int, input().split())  # Читаем количество вершин (n) и ребер (m)
    for _ in range(m):  # Читаем каждое ребро
        u, v = input().split()  # Читаем две соединенные вершины
        # Добавляем вершины в граф, если их еще нет
        if u not in G:
            G[u] = {}  # Для вершины u создаем словарь смежных вершин
        if v not in G:
            G[v] = {}  # Для вершины v создаем словарь смежных вершин  
        w = randint(1, 100)  # Генерируем случайный вес ребра от 1 до 100
        G[u][v] = w  # Добавляем ребро u -> v с весом w
        G[v][u] = w  # Добавляем ребро v -> u с тем же весом (граф неориентированный)
    return G  # Возвращаем полученный граф

from heapq import heappush, heappop  # Импортируем функции для работы с кучей (приоритетной очередью)

def deykstra(G: dict, start: str):
    """Реализация алгоритма Дейкстры для поиска кратчайших путей от начальной вершины"""
    h = []  # Инициализируем кучу (приоритетную очередь)
    heappush(h, (0, start))  # Добавляем начальную вершину с расстоянием 0
    # Инициализируем расстояния до всех вершин как бесконечность
    dist = {v: float('+inf') for v in G}
    dist[start] = 0  # Расстояние до начальной вершины = 0
    used = set()  # Множество для отслеживания посещенных вершин
    while len(h):  # Пока в куче есть элементы
        w, v = heappop(h)  # Извлекаем вершину с минимальным расстоянием
        # Проверяем всех соседей текущей вершины
        for neighbor in G[v]:
            # Если найден более короткий путь до соседа
            if w + G[v][neighbor] < dist[neighbor]:
                dist[neighbor] = w + G[v][neighbor]  # Обновляем расстояние
                heappush(h, (dist[neighbor], neighbor))  # Добавляем в кучу с новым расстоянием
        used.add(v)  # Помечаем вершину как посещенную
    return dist  # Возвращаем словарь с кратчайшими расстояниями до всех вершин

# 18. Алгоритм Флойда-Уоршелла.

Алгоритм Флойда-Уоршелла находит **кратчайшие пути между всеми парами вершин** в взвешенном графе. Работает с:
- Отрицательными весами рёбер (но без отрицательных циклов)
- Ориентированными и неориентированными графами

## **Основная идея**
Динамическое программирование: последовательно улучшаем оценки кратчайших путей, рассматривая все возможные промежуточные вершины.

### **Матрица расстояний**
- `dist[i][j]` — длина кратчайшего пути из `i` в `j`
- Изначально:
  - `dist[i][j] = 0`, если `i == j`
  - `dist[i][j] = вес ребра i→j`, если ребро существует
  - `dist[i][j] = +∞`, если пути нет

## **Сложность**
- **Время**: O(V³) — три вложенных цикла по количеству вершин
- **Память**: O(V²) — хранение матрицы расстояний

In [ ]:
def floyd_warshall(graph):
    n = len(graph)
    # Инициализация матрицы расстояний
    dist = [[float('inf')] * n for _ in range(n)]
    for i in range(n):
        dist[i][i] = 0
    for u in graph:
        for v in graph[u]:
            dist[u][v] = graph[u][v]
    # Основная часть алгоритма
    for k in range(n):           # Промежуточная вершина
        for i in range(n):       # Начальная вершина
            for j in range(n):   # Конечная вершина
                if dist[i][k] + dist[k][j] < dist[i][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
    return dist

# 19. Алгоритм Форда-Беллмана.

Алгоритм Форда-Беллмана находит **кратчайшие пути от одной стартовой вершины до всех остальных** в взвешенном графе. Ключевые особенности:
- Работает с **отрицательными весами** рёбер
- Обнаруживает **отрицательные циклы**, достижимые из стартовой вершины
- Применим для **ориентированных** и **неориентированных** графов (в неориентированном случае отрицательные рёбра автоматически создают отрицательные циклы)

## **Основная идея**
Алгоритм релаксации: последовательно улучшает оценки кратчайших путей, рассматривая все рёбра графа. После `V-1` итераций гарантированно находит оптимальные пути (если нет отрицательных циклов).

## **Обнаружение отрицательных циклов**
Алгоритм выполняет дополнительную проверку после `V-1` итераций:
- Если на `V`-й итерации происходит обновление расстояний → существует отрицательный цикл

## **Сложность**
- **Время**: O(V×E) — в худшем случае (V-1) полных проходов по всем рёбрам
- **Память**: O(V) — хранение расстояний до вершин

In [ ]:
def ford_bellman(graph, start):
    # Инициализация расстояний
    dist = {v: float('inf') for v in graph}
    dist[start] = 0
    edges = []
    # Составляем список всех рёбер
    for u in graph:
        for v, w in graph[u].items():
            edges.append((u, v, w))
    # Основная часть алгоритма
    for _ in range(len(graph) - 1):
        updated = False
        for u, v, w in edges:
            if dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                updated = True
        if not updated:  # Досрочный выход если нет изменений
            break
    # Проверка на отрицательные циклы
    for u, v, w in edges:
        if dist[u] + w < dist[v]:
            raise ValueError("Граф содержит отрицательный цикл")
    return dist

# 20. Остовные деревья. Алгоритм Прима.

**Остовное дерево** — это подмножество рёбер связного неориентированного графа, которое соединяет все вершины графа, не образуя циклов, и имеет минимальную возможную сумму весов рёбер.

**Алгоритм Прима** — это жадный алгоритм для нахождения минимального остовного дерева в графе. Он работает следующим образом:

1. Начинаем с произвольной вершины графа и добавляем её в остовное дерево.
2. На каждом шаге выбираем рёбер, которое соединяет уже добавленные в остовное дерево вершины с вершинами, которые ещё не добавлены, и имеет минимальный вес.
3. Добавляем эту вершину и соответствующее ребро в остовное дерево.
4. Повторяем шаги 2 и 3, пока все вершины не будут добавлены в остовное дерево.

### Объяснение кода

1. **Граф** представлен в виде словаря, где ключи — это вершины, а значения — это словари соседей с весами рёбер.
2. **Куча** (min_heap) используется для хранения рёбер, которые могут быть добавлены в остовное дерево, с их весами.
3. **Цикл** продолжается, пока есть рёбра в куче. Если текущая вершина уже посещена, она пропускается.
4. Если вершина не посещена, она добавляется в остовное дерево, и все её соседние рёбра добавляются в кучу, если соседние вершины ещё не посещены.
5. В конце выводится список рёбер минимального остовного дерева.

Этот алгоритм работает за \(O(E \log V)\), где \(E\) — количество рёбер, а \(V\) — количество вершин в графе.

In [ ]:
import heapq

def prim(graph, start):
    # Инициализация
    mst = []  # Список рёбер остовного дерева
    visited = set()  # Множество посещённых вершин
    min_heap = [(0, start, None)]  # (вес, текущая вершина, предыдущая вершина)
    while min_heap:
        weight, current_vertex, from_vertex = heapq.heappop(min_heap)
        if current_vertex in visited:
            continue
        visited.add(current_vertex)
        if from_vertex is not None:
            mst.append((from_vertex, current_vertex, weight))
        for neighbor, edge_weight in graph[current_vertex].items():
            if neighbor not in visited:
                heapq.heappush(min_heap, (edge_weight, neighbor, current_vertex))
    return mst

graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'C': 2, 'D': 5},
    'C': {'A': 4, 'B': 2, 'D': 1},
    'D': {'B': 5, 'C': 1}
}
start_vertex = 'A'
mst = prim(graph, start_vertex)
print("Рёбра минимального остовного дерева:")
for from_vertex, to_vertex, weight in mst:
    print(f"{from_vertex} - {to_vertex}: {weight}")

Рёбра минимального остовного дерева:
A - B: 1
B - C: 2
C - D: 1


# 21. Остовные деревья. Алгоритм Краскала. Система непересекающихся множеств для оптимизации алгоритма.

**Остовное дерево** — это подмножество рёбер связного неориентированного графа, которое соединяет все вершины графа, не образуя циклов, и имеет минимальную возможную сумму весов рёбер.

**Алгоритм Краскала** — это жадный алгоритм для нахождения минимального остовного дерева, который работает следующим образом:

1. Сначала сортируем все рёбра графа по возрастанию их веса.
2. Инициализируем пустое остовное дерево.
3. Проходим по отсортированным рёбрам и добавляем каждое ребро в остовное дерево, если оно не образует цикл с уже добавленными рёбрами.
4. Для проверки, образует ли ребро цикл, используем структуру данных "система непересекающихся множеств" (Union-Find).

### Система непересекающихся множеств

Система непересекающихся множеств — это структура данных, которая поддерживает операции объединения и поиска. Она позволяет эффективно управлять множествами и проверять, принадлежат ли два элемента одному множеству.

Основные операции:
- **Find**: находит корень (представителя) множества, к которому принадлежит элемент.
- **Union**: объединяет два множества.

Для оптимизации этих операций часто используются два метода:
- **Сжатие пути** (Path Compression) в операции Find.
- **Объединение по рангу** (Union by Rank) в операции Union.

In [ ]:
class HeapEl:
    def __init__(self, name, dist):
        self.name = name
        self.edge = None
        self.pos = -1 
        self.dist = dist

class Heap:
    def __init__(self):
        self.h = []

    def put(self, e:HeapEl):
        self.h.append(e)
        e.pos = len(self.h) - 1
        self.shift_up(e.pos)

    def get(self):
        res = self.h[0]
        res.pos = -1
        self.h[0] = self.h[-1]
        self.h[0].pos = 0
        self.h.pop()
        if len(self.h) != 0:
            self.shift_down(0)
        return res
    
    def shift_up(self, n):
        if n == 0: return
        p = (n - 1) // 2
        if self.h[n].dist < self.h[p].dist:
            self.h[n], self.h[p] = self.h[p], self.h[n]
            self.h[n].pos = n
            self.h[p].pos = p
            self.shift_up(p)

    def shift_down(self, n):
        m = self.h[n].dist 
        if 2 * n + 1 < len(self.h):
            m = min(m, self.h[2 * n + 1].dist)
        if 2 * n + 1 < len(self.h):
            m = min(m, self.h[2 * n + 2].dist)

        if m == self.h[n].dist: return
        ch = 2 * n + 1
        if m == self.h[ch].dist:
            self.h[n], self.h[ch] = self.h[ch], self.h[n]
            self.h[n].pos = n
            self.h[ch].pos = ch
            self.shift_down(ch)
            return 
        ch = 2 * n + 2
        self.h[n], self.h[ch] = self.h[ch], self.h[n]
        self.h[n].pos = n
        self.h[ch].pos = ch
        self.shift_down(ch)
        return
    
def read_graph():
    from random import randint
    N, M = map(int, input().split())
    G = {}
    for _ in range(M):
        u, v = input().split()
        w = randint(1, 20)
        if u not in G: G[u] = {}
        if v not in G: G[v] = {}
        G[u][v] = w
        G[v][u] = w

    return G

def kraskal(G):
    dist = {v: HeapEl(v, float('+inf')) for v in G}
    start = next(iter(G))
    h = Heap()

    dist[start].dist = 0
    res = [start]
    h.put(dist[start])

    while len(h.h) != 0:
        e = h.get()
        res.append(e.edge)
        res.append(e.name)
        for v in G[e.name]:
            if dist[v].dist > G[e.name][v]:
                dist[v].dist = G[e.name][v]
                dist[v].edge = f"{e.name}{v} {G[e.name][v]}"
                if dist[v].pos == -1:
                    h.put(dist[v])
                else:
                    h.shift_up(dist[v].pos)
    return res

g = read_graph()
print(kraskal(g))

In [ ]:
def read_graph():
    from random import randint
    N, M = map(int, input().slit())
    edges = []
    for _ in range(M):
        u, v = input().split()
        w = randint(1, 20)
        edges.append((w, u, v))

    return edges

parent = {}

def make_set(name):
    global parent
    parent[name] = name

def find_set(name):
    global parent
    n = name
    while parent[name] != name:
        name = parent[name]
    parent[n] = name
    return name

def union_set(a, b):
    global parent
    parent[a] = b

def prim(edges):
    global parent
    res = []
    s = 0
    edges.sort()
    for w, u, v in edges:
        if u not in parent: make_set(u)
        if v not in parent: make_set(v)
        u = find_set(u)
        v = find_set(v)
        if u != v:
            res.append(f"{u}{v}")
            union_set(u, v)
            s += w
    return res, s

e = read_graph()
print(prim(e))

# 22. Игры на ациклических графах. Решение поиском в глубину.

Игры на ациклических графах — это интересная область, в которой можно применять алгоритмы поиска в глубину (DFS) для анализа различных игровых ситуаций. Ациклические графы, такие как ориентированные ациклические графы (DAG), часто используются для моделирования игр, где нет циклов, и каждое состояние может быть достигнуто только одним способом.

### Пример игры на ациклическом графе

Рассмотрим простую игру, в которой игроки перемещаются по вершинам графа, и цель состоит в том, чтобы достичь определённой вершины (например, "выигрышной" вершины). Игроки могут перемещаться только по рёбрам графа, и игра заканчивается, когда один из игроков достигает выигрышной вершины.

### Решение с помощью поиска в глубину (DFS)

Поиск в глубину может быть использован для анализа всех возможных путей в игре. Мы можем определить, является ли текущее состояние выигрышным или проигрышным для игрока, основываясь на состоянии его соседей.

#### Алгоритм

1. Начинаем с текущей вершины.
2. Если текущая вершина — выигрышная, возвращаем "выигрыш".
3. Если все соседние вершины являются выигрышными для противника, то текущая вершина — проигрышная.
4. Если хотя бы одна соседняя вершина является проигрышной для противника, то текущая вершина — выигрышная.

### Объяснение кода

1. **Функция `dfs`**:
   - Принимает граф, текущую вершину и словарь `memo` для хранения уже вычисленных результатов.
   - Проверяет, является ли текущая вершина выигрышной или проигрышной, используя рекурсивный подход.
   - Если текущая вершина — "win", возвращает `True`.
   - Если все соседи являются выигрышными для противника, текущая вершина считается проигрышной.

2. **Граф** представлен в виде словаря, где ключи — это вершины, а значения — списки соседей.

3. **Запуск DFS** начинается с заданной стартовой вершины, и результат выводится на экран.

In [24]:
def dfs(graph, vertex, memo):
    # Если уже вычислено, возвращаем результат
    if vertex in memo:
        return memo[vertex]
    # Если текущая вершина — выигрышная
    if vertex == 'win':
        return True
    # Проверяем соседние вершины
    for neighbor in graph[vertex]:
        if not dfs(graph, neighbor, memo):  # Если сосед проигрывает
            memo[vertex] = True  # Текущая вершина выигрывает
            return True
    memo[vertex] = False  # Все соседи выигрывают, текущая проигрывает
    return False
# Пример графа
graph = {
    'A': ['B', 'C'],
    'B': ['D'],
    'C': ['D'],
    'D': ['win'],
    'E': ['F'],
    'F': ['win'],
    'win': []
}
memo = {}
start_vertex = 'A'
is_winning = dfs(graph, start_vertex, memo)
if is_winning:
    print(f"Вершина '{start_vertex}' является выигрышной.")
else:
    print(f"Вершина '{start_vertex}' является проигрышной.")

Вершина 'A' является проигрышной.


# 23. Сумма игр. Функция Шпрага-Гранди.

**Сумма игр** — это концепция в теории игр, которая позволяет анализировать ситуации, когда игроки могут выбирать из нескольких независимых игр. Если у нас есть несколько игр, то результат (выигрыш или проигрыш) зависит от состояния каждой из этих игр.

**Функция Шпрага-Гранди** используется для анализа игр с нулевой суммой, чтобы определить, является ли данная позиция выигрышной или проигрышной. Она основана на принципе, что позиция является выигрышной, если существует хотя бы один ход, который переводит её в проигрышную позицию для противника.

1. **Функция Гранди** для позиции \( G \) определяется как минимальное неотрицательное целое число, которое не может быть получено из значений функции Гранди для всех возможных ходов из позиции \( G \). Это значение называется **mex** (minimum excludant).

2. Если функция Гранди для позиции равна 0, то это проигрышная позиция для текущего игрока. Если она больше 0, то это выигрышная позиция.

Рассмотрим простую игру, в которой игроки могут удалять 1, 2 или 3 камня из кучи. Цель состоит в том, чтобы заставить противника оказаться в позиции, где он не может сделать ход.

1. **Функция `grundy(n, memo)`**:
   - Принимает количество камней \( n \) и словарь `memo` для хранения уже вычисленных значений функции Гранди.
   - Если значение уже вычислено, оно возвращается из `memo`.
   - Для каждого возможного хода (удаление 1, 2 или 3 камней) вычисляется функция Гранди для новой позиции.
   - Собирается множество `reachable`, которое содержит значения функции Гранди для всех возможных ходов.
   - Находится минимальное неотрицательное целое число, которое не содержится в `reachable` (mex), и сохраняется в `memo`.

2. **Функция `is_winning_position(n)`**:
   - Вызывает функцию `grundy` и проверяет, является ли позиция выигрышной (функция Гранди не равна 0).

In [25]:
def grundy(n, memo):
    if n in memo:
        return memo[n]
    reachable = set()
    for move in range(1, 4):  # Игрок может удалить 1, 2 или 3 камня
        if n - move >= 0:
            reachable.add(grundy(n - move, memo))
    # Находим минимальное неотрицательное целое число, которое не содержится в reachable
    mex = 0
    while mex in reachable:
        mex += 1
    memo[n] = mex
    return mex

def is_winning_position(n):
    memo = {}
    return grundy(n, memo) != 0

n = 10  # Количество камней
if is_winning_position(n):
    print(f"Позиция с {n} камнями является выигрышной.")
else:
    print(f"Позиция с {n} камнями является проигрышной.")

Позиция с 10 камнями является выигрышной.


# 24. Игры на произвольных графах.

In [ ]:
from collections import deque

def determine_winner(N, M, K, edges):
    # Строим обратный граф (куда ведут рёбра)
    adj = [[] for _ in range(N)]
    reverse_adj = [[] for _ in range(N)]
    out_degree = [0] * N
    for u, v in edges:
        adj[u].append(v)
        reverse_adj[v].append(u)
        out_degree[u] += 1
    # Инициализация: вершины без исходящих рёбер — LOSE
    status = [None] * N
    queue = deque()
    for u in range(N):
        if out_degree[u] == 0:
            status[u] = 'LOSE'
            queue.append(u)
    # Обратный BFS для пометок вершин
    while queue:
        u = queue.popleft()
        for v in reverse_adj[u]:
            if status[v] is not None:
                continue
            # Если хотя бы один переход ведёт в LOSE, вершина — WIN
            if status[u] == 'LOSE':
                status[v] = 'WIN'
                queue.append(v)
            else:
                # Уменьшаем счётчик исходящих рёбер
                out_degree[v] -= 1
                if out_degree[v] == 0:
                    status[v] = 'LOSE'
                    queue.append(v)
                    
    if status[K] == 'WIN':
        return "First"
    elif status[K] == 'LOSE':
        return "Second"
    else:
        return "Draw"

N, M, K = map(int, input().split())
edges = [tuple(map(int, input().split())) for _ in range(M)]
print(determine_winner(N, M, K, edges))

# 25. ООП, инкапсуляция, наследование, полиморфизм.
Объектно-ориентированное программирование (ООП) — это парадигма программирования, основанная на концепции "объектов", которые могут содержать данные и код: данные в виде полей (или атрибутов), а код в виде процедур (или методов). Основные принципы ООП включают инкапсуляцию, наследование и полиморфизм.

### 1. Инкапсуляция

**Инкапсуляция** — это принцип, который позволяет скрыть внутренние детали реализации объекта и предоставить доступ к ним только через определённые методы. Это помогает защитить данные от некорректного использования и упрощает взаимодействие с объектом.

### 2. Наследование

**Наследование** — это механизм, который позволяет создавать новый класс на основе существующего. Новый класс (наследник) наследует свойства и методы родительского класса (базового класса), что позволяет повторно использовать код и расширять функциональность.

### 3. Полиморфизм

**Полиморфизм** — это способность объектов разных классов обрабатывать данные по одному и тому же интерфейсу. Это позволяет использовать один и тот же метод для объектов разных классов, что упрощает код и делает его более гибким.

In [ ]:
# инкапсуляция 
class BankAccount:
    def __init__(self, owner, balance=0):
        self.owner = owner
        self.__balance = balance  # Приватное поле

    def deposit(self, amount):
        if amount > 0:
            self.__balance += amount
            print(f"Deposited {amount}. New balance: {self.__balance}")
        else:
            print("Deposit amount must be positive.")

    def withdraw(self, amount):
        if 0 < amount <= self.__balance:
            self.__balance -= amount
            print(f"Withdrew {amount}. New balance: {self.__balance}")
        else:
            print("Invalid withdrawal amount.")

    def get_balance(self):
        return self.__balance

# Пример использования
account = BankAccount("Alice", 100)
account.deposit(50)
account.withdraw(30)
print(account.get_balance())

# наследование
class Animal:
    def speak(self):
        raise NotImplementedError("Subclasses must implement this method.")

class Dog(Animal):
    def speak(self):
        return "Woof!"

class Cat(Animal):
    def speak(self):
        return "Meow!"

# Пример использования
animals = [Dog(), Cat()]
for animal in animals:
    print(animal.speak())

# полиморфизм
class Shape:
    def area(self):
        raise NotImplementedError("Subclasses must implement this method.")

class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius

    def area(self):
        return 3.14 * self.radius ** 2

shapes = [Rectangle(5, 10), Circle(7)]
for shape in shapes:
    print(f"Area: {shape.area()}")

# 26. Классы в Python. Магические методы классов.
В Python классы являются основным строительным блоком объектно-ориентированного программирования. Они позволяют создавать объекты, которые могут содержать как данные (атрибуты), так и методы (функции, связанные с объектом). 

### Магические методы классов

**Магические методы** (или специальные методы) — это методы, которые имеют специальные имена и позволяют определять поведение объектов класса в различных ситуациях. Они начинаются и заканчиваются двойными подчеркиваниями (`__`). Вот некоторые из наиболее часто используемых магических методов:

1. **`__init__`**: Конструктор класса, вызывается при создании нового объекта.
2. **`__str__`**: Определяет поведение функции `str()` и функции `print()`, возвращая строковое представление объекта.
3. **`__repr__`**: Определяет поведение функции `repr()`, возвращая официальное строковое представление объекта, которое можно использовать для его воссоздания.
4. **`__len__`**: Определяет поведение функции `len()`, возвращая длину объекта.
5. **`__getitem__`**: Позволяет использовать квадратные скобки для доступа к элементам объекта (например, `obj[key]`).
6. **`__setitem__`**: Позволяет использовать квадратные скобки для установки значений (например, `obj[key] = value`).
7. **`__delitem__`**: Позволяет использовать квадратные скобки для удаления элементов (например, `del obj[key]`).
8. **`__add__`**: Определяет поведение оператора `+`.
9. **`__sub__`**: Определяет поведение оператора `-`.
10. **`__eq__`**: Определяет поведение оператора `==`.

In [ ]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return f"Vector({self.x}, {self.y})"

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __len__(self):
        return int((self.x**2 + self.y**2) ** 0.5)

# Пример использования
v1 = Vector(2, 3)
v2 = Vector(4, 5)

print(v1)  # Вывод: Vector(2, 3)
print(v2)  # Вывод: Vector(4, 5)

v3 = v1 + v2
print(v3)  # Вывод: Vector(6, 8)

v4 = v2 - v1
print(v4)  # Вывод: Vector(2, 2)

print(v1 == v2)  # Вывод: False
print(len(v1))   # Вывод: 3

# 27. Абстрактные классы, модуль abc.
Абстрактные классы в Python — это классы, которые не могут быть инстанцированы напрямую и предназначены для определения интерфейса для других классов. Они позволяют задавать методы, которые должны быть реализованы в дочерних классах, обеспечивая тем самым определённый уровень абстракции.

Для создания абстрактных классов в Python используется модуль `abc` (Abstract Base Classes). Этот модуль предоставляет инструменты для определения абстрактных классов и абстрактных методов.

### Основные компоненты модуля `abc`

1. **`ABC`**: Базовый класс для определения абстрактных классов.
2. **`abstractmethod`**: Декоратор, который используется для определения абстрактных методов.

### Объяснение кода

1. **Импортирование модуля**: Мы импортируем `ABC` и `abstractmethod` из модуля `abc`.

2. **Определение абстрактного класса**: Класс `Shape` наследует от `ABC` и содержит два абстрактных метода: `area` и `perimeter`. Эти методы не имеют реализации и должны быть реализованы в дочерних классах.

3. **Создание дочерних классов**: Классы `Rectangle` и `Circle` наследуют от `Shape` и реализуют абстрактные методы `area` и `perimeter`.

4. **Использование классов**: Мы создаем список фигур и для каждой фигуры вызываем методы `area` и `perimeter`, которые возвращают соответствующие значения.

### Преимущества использования абстрактных классов

- **Определение интерфейса**: Абстрактные классы позволяют определить интерфейс, который должны реализовать все дочерние классы, что обеспечивает согласованность.
- **Упрощение кода**: Они помогают избежать дублирования кода, так как общая логика может быть реализована в абстрактном классе.
- **Улучшение читаемости**: Код становится более понятным, так как явно указывает, какие методы должны быть реализованы.

In [ ]:
from abc import ABC, abstractmethod

class AbstractMonopod(ABC):
    def __init__(self, name):
        self.name = name

    @abstractmethod
    def get_hello_text(self):
        pass

    @abstractmethod
    def make_magic(self):
        pass

class InvisibleMonopod(AbstractMonopod):
    status = "invisible"

    def get_hello_text(self):
        return f"Hello, my name is {self.name}. I'm {self.status.capitalize()} squared monopod."

    def make_magic(self, number):
        return number ** 2  


class CuteMonopod(AbstractMonopod):
    status = "cute" 

    def get_hello_text(self):
        return f"Hello, my name is {self.name}. I'm {self.status.capitalize()} negative monopod."

    def make_magic(self, number):
        return -number
    
class MyStaticMethod:
    def __init__(self, func):
        """
        Инициализация статического метода.
        :param func: Функция, которую нужно сделать статическим методом.
        """
        self.func = func

    def __get__(self, obj, objtype=None):
        """
        Вызывается при доступе к методу через класс или экземпляр.
        :param obj: Экземпляр класса (если вызывается через экземпляр).
        :param objtype: Класс (если вызывается через класс).
        :return: Саму функцию без привязки к экземпляру или классу.
        """
        return self.func

# 28. Генераторы и сопроцессы
Генераторы и сопроцессы — это мощные инструменты в Python, которые позволяют работать с итерациями и асинхронным программированием. Давайте рассмотрим каждую из этих концепций подробнее.

**Генераторы** — это специальные функции, которые позволяют создавать итераторы. Они используют ключевое слово `yield` для возврата значений по одному за раз, сохраняя состояние функции между вызовами. Это позволяет экономить память, так как значения генерируются по мере необходимости, а не создаются все сразу.

1. **Определение генератора**: Функция `fibonacci` определяет генератор, который генерирует первые `n` чисел Фибоначчи.
2. **Использование `yield`**: Вместо `return` используется `yield`, что позволяет функции возвращать значение и сохранять своё состояние.
3. **Итерация**: При итерации по генератору значения генерируются по одному, что позволяет экономить память.

**Сопроцессы** (или корутины) — это более сложная концепция, которая позволяет выполнять несколько функций одновременно, при этом они могут приостанавливать своё выполнение и передавать управление другим функциям. В Python сопроцессы реализуются с помощью `async` и `await`.


- **Генераторы**: Используются для создания итераторов, которые возвращают значения по одному. Они не могут приостанавливать выполнение и возвращать управление другим функциям.
- **Сопроцессы**: Позволяют выполнять несколько функций одновременно, приостанавливая выполнение и передавая управление другим функциям. Они идеально подходят для асинхронного программирования.

In [ ]:
def range(start=0, stop=None, step=1):
    if stop is None:
        stop = start
        start = 0
    if step > 0:
        while start < stop:
            yield start
            start += step
    elif step < 0: 
        while start > stop:
            yield start
            start += step

def enumerate(Iterated, start_index=0):
    for el in Iterated:
        yield start_index, el
        start_index += 1

def zip(*args):
    min_length = min(len(arg) for arg in args)
    for i in range(min_length):
        yield tuple(arg[i] for arg in args)

# 29. Декораторы

Декораторы в Python — это мощный инструмент, который позволяет изменять или расширять поведение функций или методов без изменения их кода. Декораторы представляют собой функции, которые принимают другую функцию в качестве аргумента и возвращают новую функцию, которая обычно добавляет некоторую функциональность.

### Основы декораторов

Декораторы часто используются для:

- Логирования
- Измерения времени выполнения
- Проверки прав доступа
- Кэширования результатов

1. **Определение декоратора**: Функция `my_decorator` принимает функцию `func` в качестве аргумента и определяет внутреннюю функцию `wrapper`, которая добавляет дополнительное поведение.
2. **Использование `@`**: Декоратор применяется к функции `say_hello` с помощью символа `@`. Это эквивалентно вызову `say_hello = my_decorator(say_hello)`.
3. **Вызов функции**: При вызове `say_hello()` фактически вызывается `wrapper()`, который добавляет логирование до и после вызова оригинальной функции.

### Декораторы с аргументами

Декораторы могут также принимать аргументы. Для этого нужно создать дополнительный уровень вложенности:

1. **Декоратор с аргументами**: Функция `repeat` принимает аргумент `num_times` и возвращает декоратор `decorator_repeat`.
2. **Внутренний `wrapper`**: Внутренний `wrapper` вызывает оригинальную функцию `func` указанное количество раз.
3. **Применение декоратора**: Декоратор применяется к функции `greet`, и при её вызове она будет выполнена трижды.

In [28]:
def swap(func):
    def reverse(a, b):
        return func(b, a)
    return reverse

def swap(func):
    def new_func(*args, **kwargs):
        swap = list(args)
        for i in range(0, len(swap) - 1, 2):
            swap[i], swap[i + 1] = swap[i + 1], swap[i]
        return func(*swap, **kwargs)
    return new_func

def debug(form):
    def decorator(func):
        def new_fun(*args, **kwargs):
            res = func(*args, **kwargs)
            debug_info = form.format(
                name=func.__name__,
                args=args,
                kwargs=kwargs,
                res=res
            )
            print(debug_info)
            return res
        return new_fun
    return decorator

# Extra 
# 1. Ассинхронное программирование

Асинхронное программирование — это парадигма, которая позволяет выполнять операции, не блокируя основной поток выполнения программы. Это особенно полезно в ситуациях, когда необходимо выполнять длительные операции, такие как ввод-вывод (I/O), сетевые запросы или работа с базами данных, не задерживая выполнение других задач.

### Основные концепции асинхронного программирования

1. **Асинхронные функции**: Функции, которые могут приостанавливать своё выполнение и возвращать управление другим функциям. В Python они определяются с помощью ключевого слова `async`.

2. **Ожидание (await)**: Ключевое слово `await` используется для приостановки выполнения асинхронной функции до тех пор, пока не завершится выполнение другой асинхронной функции.

3. **Сопроцессы (coroutines)**: Это функции, которые могут быть приостановлены и возобновлены. В Python сопроцессы создаются с помощью `async def`.

4. **Цикл событий (event loop)**: Это механизм, который управляет выполнением асинхронных задач. Он обрабатывает события и запускает сопроцессы.

### Пример асинхронного программирования

Вот простой пример, который демонстрирует использование асинхронного программирования в Python с помощью модуля `asyncio`:

```python
import asyncio

async def fetch_data():
    print("Начинаем загрузку данных...")
    await asyncio.sleep(2)  # Симуляция длительной операции
    print("Данные загружены!")
    return {"data": 42}

async def main():
    print("Запуск основной функции...")
    data = await fetch_data()  # Ожидание завершения fetch_data
    print(f"Полученные данные: {data}")

# Запуск асинхронной программы
asyncio.run(main())
```

### Объяснение кода

1. **Определение асинхронной функции**: Функция `fetch_data` определена как асинхронная с помощью `async def`. Она использует `await` для приостановки выполнения на 2 секунды, имитируя длительную операцию.

2. **Основная функция**: Функция `main` также является асинхронной и вызывает `fetch_data`, ожидая её завершения.

3. **Запуск программы**: `asyncio.run(main())` запускает цикл событий и выполняет основную функцию.

### Преимущества асинхронного программирования

- **Не блокирует выполнение**: Асинхронные операции позволяют выполнять другие задачи, пока ожидается завершение длительных операций.
- **Эффективное использование ресурсов**: Асинхронное программирование позволяет более эффективно использовать ресурсы, особенно в приложениях, которые требуют высокой производительности и отзывчивости.
- **Упрощение кода**: Асинхронные функции могут сделать код более читаемым и понятным по сравнению с использованием потоков или процессов.

### Заключение

Асинхронное программирование в Python предоставляет мощные инструменты для работы с длительными операциями, позволяя создавать более отзывчивые и эффективные приложения. Использование `async` и `await` в сочетании с модулем `asyncio` позволяет легко управлять асинхронными задачами и упрощает написание кода, который работает с вводом-выводом и сетевыми запросами.

In [ ]:
import asyncio

async def fetch_data():
    print("Начинаем загрузку данных...")
    await asyncio.sleep(2)  # Симуляция длительной операции
    print("Данные загружены!")
    return {"data": 42}

async def main():
    print("Запуск основной функции...")
    data = await fetch_data()  # Ожидание завершения fetch_data
    print(f"Полученные данные: {data}")

# Запуск асинхронной программы
asyncio.run(main())

# 2. Многопоточное программирование, библиотека threading
Многопоточное программирование — это парадигма, позволяющая выполнять несколько потоков (или "нитей") одновременно в рамках одного процесса. Это может быть полезно для выполнения параллельных задач, таких как обработка данных, выполнение сетевых запросов или выполнение длительных операций, не блокируя основной поток выполнения программы.

### Библиотека `threading`

В Python для работы с потоками используется стандартная библиотека `threading`. Она предоставляет высокоуровневый интерфейс для создания и управления потоками.

### Основные компоненты библиотеки `threading`

1. **Thread**: Класс, представляющий поток. Вы можете создать новый поток, передав целевую функцию и её аргументы.
2. **Lock**: Объект, который используется для синхронизации потоков, предотвращая одновременный доступ к общим ресурсам.
3. **Event**: Объект, который позволяет потокам сигнализировать друг другу о событиях.
4. **Condition**: Объект, который позволяет потокам ожидать определённых условий.

### Пример использования библиотеки `threading`

Вот простой пример, который демонстрирует создание и запуск потоков с использованием библиотеки `threading`:

```python
import threading
import time

def worker(thread_id):
    print(f"Поток {thread_id} начал работу.")
    time.sleep(2)  # Симуляция длительной работы
    print(f"Поток {thread_id} завершил работу.")

# Создание списка потоков
threads = []
for i in range(5):
    thread = threading.Thread(target=worker, args=(i,))
    threads.append(thread)
    thread.start()  # Запуск потока

# Ожидание завершения всех потоков
for thread in threads:
    thread.join()

print("Все потоки завершили работу.")
```

### Объяснение кода

1. **Импортирование библиотеки**: Мы импортируем библиотеку `threading` и `time` для использования функции `sleep`.

2. **Определение функции `worker`**: Эта функция принимает идентификатор потока и выполняет некоторую работу (в данном случае просто ждет 2 секунды).

3. **Создание и запуск потоков**:
   - Мы создаем список `threads` для хранения потоков.
   - В цикле создаем 5 потоков, передавая функции `worker` идентификатор потока в качестве аргумента.
   - Каждый поток запускается с помощью метода `start()`.

4. **Ожидание завершения потоков**: Мы используем метод `join()`, чтобы дождаться завершения всех потоков перед тем, как продолжить выполнение основной программы.

### Синхронизация потоков

При работе с многопоточностью важно учитывать, что несколько потоков могут одновременно пытаться получить доступ к общим ресурсам, что может привести к состояниям гонки. Для предотвращения этого можно использовать объекты `Lock`.

Вот пример использования `Lock` для синхронизации потоков:

```python
import threading

# Общий ресурс
counter = 0
lock = threading.Lock()

def increment():
    global counter
    for _ in range(100000):
        with lock:  # Захват блокировки
            counter += 1  # Изменение общего ресурса

# Создание потоков
threads = []
for _ in range(2):
    thread = threading.Thread(target=increment)
    threads.append(thread)
    thread.start()

# Ожидание завершения всех потоков
for thread in threads:
    thread.join()

print(f"Итоговое значение счетчика: {counter}")
```

### Объяснение кода

1. **Общий ресурс**: Мы определяем переменную `counter`, к которой будут обращаться потоки, и создаем объект `Lock`.

2. **Функция `increment`**: Эта функция увеличивает значение `counter` 100000 раз. Блокировка (`lock`) используется для обеспечения того, что только один поток может изменять `counter` в любой момент времени.

3. **Создание и запуск потоков**: Мы создаем два потока, которые выполняют функцию `increment`.

4. **Ожидание завершения потоков**: Мы используем `join()`, чтобы дождаться завершения всех потоков.

### Заключение

Многопоточное программирование в Python с использованием библиотеки `threading` позволяет эффективно выполнять параллельные задачи. Однако важно учитывать проблемы синхронизации и состояния гонки, которые могут возникнуть при работе с общими ресурсами. Использование объектов `Lock` и других механизмов синхронизации помогает избежать этих проблем и обеспечивает корректное выполнение многопоточных программ.

In [ ]:
import threading
import time

def worker(thread_id):
    print(f"Поток {thread_id} начал работу.")
    time.sleep(2)  # Симуляция длительной работы
    print(f"Поток {thread_id} завершил работу.")

# Создание списка потоков
threads = []
for i in range(5):
    thread = threading.Thread(target=worker, args=(i,))
    threads.append(thread)
    thread.start()  # Запуск потока

# Ожидание завершения всех потоков
for thread in threads:
    thread.join()

print("Все потоки завершили работу.")

import threading

# Общий ресурс
counter = 0
lock = threading.Lock()

def increment():
    global counter
    for _ in range(100000):
        with lock:  # Захват блокировки
            counter += 1  # Изменение общего ресурса

# Создание потоков
threads = []
for _ in range(2):
    thread = threading.Thread(target=increment)
    threads.append(thread)
    thread.start()

# Ожидание завершения всех потоков
for thread in threads:
    thread.join()

print(f"Итоговое значение счетчика: {counter}")


# 3. Параллельное программирование, библиотека multiprocessing
Параллельное программирование — это парадигма, позволяющая выполнять несколько процессов одновременно, что может значительно ускорить выполнение задач, особенно на многоядерных процессорах. В Python для реализации параллельного программирования используется библиотека `multiprocessing`, которая позволяет создавать отдельные процессы, каждый из которых имеет собственное пространство памяти.

### Основные компоненты библиотеки `multiprocessing`

1. **Process**: Класс, представляющий отдельный процесс. Вы можете создать новый процесс, передав целевую функцию и её аргументы.
2. **Queue**: Объект, который позволяет обмениваться данными между процессами.
3. **Pipe**: Объект, который позволяет устанавливать двустороннюю связь между двумя процессами.
4. **Pool**: Позволяет создавать пул процессов для выполнения задач параллельно, что упрощает распределение работы.

### Пример использования библиотеки `multiprocessing`

Вот простой пример, который демонстрирует создание и запуск процессов с использованием библиотеки `multiprocessing`:

```python
import multiprocessing
import time

def worker(process_id):
    print(f"Процесс {process_id} начал работу.")
    time.sleep(2)  # Симуляция длительной работы
    print(f"Процесс {process_id} завершил работу.")

if __name__ == "__main__":
    # Создание списка процессов
    processes = []
    for i in range(5):
        process = multiprocessing.Process(target=worker, args=(i,))
        processes.append(process)
        process.start()  # Запуск процесса

    # Ожидание завершения всех процессов
    for process in processes:
        process.join()

    print("Все процессы завершили работу.")
```

### Объяснение кода

1. **Импортирование библиотеки**: Мы импортируем библиотеку `multiprocessing` и `time` для использования функции `sleep`.

2. **Определение функции `worker`**: Эта функция принимает идентификатор процесса и выполняет некоторую работу (в данном случае просто ждет 2 секунды).

3. **Создание и запуск процессов**:
   - Мы создаем список `processes` для хранения процессов.
   - В цикле создаем 5 процессов, передавая функции `worker` идентификатор процесса в качестве аргумента.
   - Каждый процесс запускается с помощью метода `start()`.

4. **Ожидание завершения процессов**: Мы используем метод `join()`, чтобы дождаться завершения всех процессов перед тем, как продолжить выполнение основной программы.

### Использование `Pool` для параллельного выполнения задач

Библиотека `multiprocessing` также предоставляет класс `Pool`, который позволяет создавать пул процессов и распределять задачи между ними. Это упрощает выполнение большого количества однотипных задач.

Вот пример использования `Pool`:

```python
import multiprocessing
import time

def square(n):
    time.sleep(1)  # Симуляция длительной работы
    return n * n

if __name__ == "__main__":
    numbers = [1, 2, 3, 4, 5]
    
    # Создание пула процессов
    with multiprocessing.Pool(processes=3) as pool:
        results = pool.map(square, numbers)  # Параллельное выполнение функции square

    print("Результаты:", results)
```

### Объяснение кода

1. **Определение функции `square`**: Эта функция принимает число и возвращает его квадрат, имитируя длительную работу с помощью `sleep`.

2. **Создание пула процессов**: Мы создаем пул из 3 процессов с помощью `multiprocessing.Pool`.

3. **Параллельное выполнение**: Метод `map` используется для параллельного выполнения функции `square` для каждого элемента в списке `numbers`. Результаты собираются в список `results`.

4. **Вывод результатов**: После завершения всех процессов выводятся результаты.

### Заключение

Параллельное программирование с использованием библиотеки `multiprocessing` в Python позволяет эффективно использовать многоядерные процессоры для выполнения задач. Это особенно полезно для CPU-bound задач, таких как вычисления, где время выполнения может быть значительно сокращено за счет параллельного выполнения. Библиотека `multiprocessing` предоставляет удобные инструменты для создания процессов, обмена данными и управления выполнением задач.

In [ ]:
import multiprocessing
import time

def worker(process_id):
    print(f"Процесс {process_id} начал работу.")
    time.sleep(2)  # Симуляция длительной работы
    print(f"Процесс {process_id} завершил работу.")

if __name__ == "__main__":
    # Создание списка процессов
    processes = []
    for i in range(5):
        process = multiprocessing.Process(target=worker, args=(i,))
        processes.append(process)
        process.start()  # Запуск процесса

    # Ожидание завершения всех процессов
    for process in processes:
        process.join()

    print("Все процессы завершили работу.")

import multiprocessing
import time

def square(n):
    time.sleep(1)  # Симуляция длительной работы
    return n * n

if __name__ == "__main__":
    numbers = [1, 2, 3, 4, 5]
    
    # Создание пула процессов
    with multiprocessing.Pool(processes=3) as pool:
        results = pool.map(square, numbers)  # Параллельное выполнение функции square

    print("Результаты:", results)
